Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [7]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [8]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [9]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [10]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

Solution: 
CASE : Only NN (Input/Output) (with SGD)

Same steps followed from Assignment 2 but loss function is added with the regularization term nn.l2_loss(t)

In [16]:
batch_size = 128
regularization_parameter = 0.001

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) + regularization_parameter * tf.nn.l2_loss(weights)
  
  
  # ReLu Layer.
  ReLu = tf.nn.relu(tf.matmul(tf_train_dataset, weights) + biases)


  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [18]:
num_steps = 8001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 22.732229
Minibatch accuracy: 10.2%
Validation accuracy: 10.4%
Minibatch loss at step 500: 2.310347
Minibatch accuracy: 82.8%
Validation accuracy: 76.9%
Minibatch loss at step 1000: 1.494843
Minibatch accuracy: 80.5%
Validation accuracy: 79.8%
Minibatch loss at step 1500: 1.315019
Minibatch accuracy: 81.2%
Validation accuracy: 80.5%
Minibatch loss at step 2000: 0.984161
Minibatch accuracy: 82.0%
Validation accuracy: 82.2%
Minibatch loss at step 2500: 0.717950
Minibatch accuracy: 85.9%
Validation accuracy: 82.8%
Minibatch loss at step 3000: 0.930256
Minibatch accuracy: 80.5%
Validation accuracy: 82.8%
Minibatch loss at step 3500: 0.542333
Minibatch accuracy: 86.7%
Validation accuracy: 82.9%
Minibatch loss at step 4000: 0.795056
Minibatch accuracy: 81.2%
Validation accuracy: 82.8%
Minibatch loss at step 4500: 0.873934
Minibatch accuracy: 82.0%
Validation accuracy: 82.3%
Minibatch loss at step 5000: 0.602939
Minibatch accuracy: 85.9%
Validation accura

CASE : Only Hidden layer (with SGD)

Same steps followed from Assignment 2 but loss function is added with the regularization term nn.l2_loss(t)

In [20]:
#Prasannas_Code
batch_size = 128
hidden_nodes = 1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  

  # -----------layer 1(input X) ---------relu --------layer 2 (output classes)
  # ----weights_input,biases_input ------relu_nodes----weights_output,biases_output 

  weights_input = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_nodes]))
  biases_input = tf.Variable(tf.zeros([hidden_nodes]))
  
  # ReLu Layer.
  ReLu = tf.nn.relu(tf.matmul(tf_train_dataset, weights_input) + biases_input)
    
  weights_output = tf.Variable(tf.truncated_normal([hidden_nodes, num_labels]))
  biases_output = tf.Variable(tf.zeros([num_labels]))

  logits = tf.matmul(ReLu, weights_output) + biases_output

  #defining loss function
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) + regularization_parameter * ( tf.nn.l2_loss(weights_input)
+ tf.nn.l2_loss(weights_output))

  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights_input) + biases_input), weights_output) + biases_output)
  test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights_input) + biases_input), weights_output) + biases_output)

In [21]:
#Prasannas_Code
num_steps = 8001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):

    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]

    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 584.730225
Minibatch accuracy: 21.1%
Validation accuracy: 35.7%
Minibatch loss at step 500: 188.294601
Minibatch accuracy: 85.9%
Validation accuracy: 81.6%
Minibatch loss at step 1000: 113.627357
Minibatch accuracy: 84.4%
Validation accuracy: 81.0%
Minibatch loss at step 1500: 69.415932
Minibatch accuracy: 83.6%
Validation accuracy: 82.7%
Minibatch loss at step 2000: 41.559868
Minibatch accuracy: 84.4%
Validation accuracy: 85.0%
Minibatch loss at step 2500: 25.093266
Minibatch accuracy: 90.6%
Validation accuracy: 86.7%
Minibatch loss at step 3000: 15.478595
Minibatch accuracy: 88.3%
Validation accuracy: 87.5%
Minibatch loss at step 3500: 9.440487
Minibatch accuracy: 92.2%
Validation accuracy: 87.9%
Minibatch loss at step 4000: 6.049778
Minibatch accuracy: 90.6%
Validation accuracy: 88.2%
Minibatch loss at step 4500: 3.841372
Minibatch accuracy: 88.3%
Validation accuracy: 88.6%
Minibatch loss at step 5000: 2.498961
Minibatch accuracy: 88.3%
Validati

---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

Solution: 

Initial Guess by theory: Since the model is trained for few batches in SGD , the model fits only for the training data. This might lead to poor results in test/validation.

In [57]:
#Prasannas_Code
batch_size = 128
hidden_nodes = 1024
regularization_parameter = 0
graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  

  # -----------layer 1(input X) ---------relu --------layer 2 (output classes)
  # ----weights_input,biases_input ------relu_nodes----weights_output,biases_output 

  weights_input = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_nodes]))
  biases_input = tf.Variable(tf.zeros([hidden_nodes]))
  
  # ReLu Layer.
  ReLu = tf.nn.relu(tf.matmul(tf_train_dataset, weights_input) + biases_input)
    
  weights_output = tf.Variable(tf.truncated_normal([hidden_nodes, num_labels]))
  biases_output = tf.Variable(tf.zeros([num_labels]))

  logits = tf.matmul(ReLu, weights_output) + biases_output

  #defining loss function
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) + regularization_parameter * ( tf.nn.l2_loss(weights_input)
+ tf.nn.l2_loss(weights_output))

  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights_input) + biases_input), weights_output) + biases_output)
  test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights_input) + biases_input), weights_output) + biases_output)

In [67]:
#Prasannas_Code
num_steps = 8001
number_of_batches = 5
with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):

    offset = ((step%number_of_batches) * batch_size) % (train_labels.shape[0] - batch_size)
    print(offset)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]

    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
0
Minibatch loss at step 0: 354.090637
Minibatch accuracy: 12.5%
Validation accuracy: 35.9%
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
51

512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128


384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
Minibatch loss at step 4500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 77.0%
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512


384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0
128
256
384
512
0


You could see that our training data is always the same dataset i.e. offset gives same data range 0-128,256,384,512. So if we train again and again with the same dataset with more iterations, obviously it will get overfitted as expected. Because its overfitting with minibatch, our training data accuracy is poor.

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

Solution: 

Initial Guess by theory: Accuracy should get improved by introducing dropout layers.


In [72]:
#Prasannas_Code
batch_size = 128
hidden_nodes = 1024
regularization_parameter = 0
graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  

  # -----------layer 1(input X) ---------relu --------layer 2 (output classes)
  # ----weights_input,biases_input ------relu_nodes----weights_output,biases_output 

  weights_input = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_nodes]))
  biases_input = tf.Variable(tf.zeros([hidden_nodes]))
  
  # ReLu Layer.
  ReLu = tf.nn.relu(tf.matmul(tf_train_dataset, weights_input) + biases_input)

  #dropout layer
  dropOut = tf.nn.dropout(ReLu, 0.4)
    
  weights_output = tf.Variable(tf.truncated_normal([hidden_nodes, num_labels]))
  biases_output = tf.Variable(tf.zeros([num_labels]))

  logits = tf.matmul(dropOut, weights_output) + biases_output

  #defining loss function
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) + regularization_parameter * ( tf.nn.l2_loss(weights_input)
+ tf.nn.l2_loss(weights_output))

  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights_input) + biases_input), weights_output) + biases_output)
  test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights_input) + biases_input), weights_output) + biases_output)

In [73]:
#Prasannas_Code
num_steps = 8001
number_of_batches = 5
with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):

    offset = ((step%number_of_batches) * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]

    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 484.957153
Minibatch accuracy: 14.1%
Validation accuracy: 28.4%
Minibatch loss at step 500: 0.346389
Minibatch accuracy: 98.4%
Validation accuracy: 78.9%
Minibatch loss at step 1000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 79.1%
Minibatch loss at step 1500: 0.146649
Minibatch accuracy: 99.2%
Validation accuracy: 78.9%
Minibatch loss at step 2000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 79.0%
Minibatch loss at step 2500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 79.0%
Minibatch loss at step 3000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 79.2%
Minibatch loss at step 3500: 0.019130
Minibatch accuracy: 99.2%
Validation accuracy: 79.2%
Minibatch loss at step 4000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 78.7%
Minibatch loss at step 4500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 78.7%
Minibatch loss at step 5000: 0.000000
Minibatch accuracy: 100.0%
Validatio

In [ ]:
Dropout has slightly increased test accuracy eventhough our model is overfitting with training data.

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [96]:
#Prasannas_Code
batch_size = 700
hidden_nodes = 2048
regularization_parameter = 0.001
graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  

  # -----------layer 1(input X) ---------relu --------layer 2 (output classes)
  # ----weights_input,biases_input ------relu_nodes----weights_output,biases_output 

  weights_input = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_nodes]))
  biases_input = tf.Variable(tf.zeros([hidden_nodes]))
  
  # ReLu Layer.
  ReLu = tf.nn.relu(tf.matmul(tf_train_dataset, weights_input) + biases_input)

  #dropout layer
  dropOut = tf.nn.dropout(ReLu, 0.4)
    
  weights_output = tf.Variable(tf.truncated_normal([hidden_nodes, num_labels]))
  biases_output = tf.Variable(tf.zeros([num_labels]))

  logits = tf.matmul(dropOut, weights_output) + biases_output

  #defining loss function
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) + regularization_parameter * ( tf.nn.l2_loss(weights_input)
+ tf.nn.l2_loss(weights_output))

  # Optimizer.
  global_step = tf.Variable(0) 
  learning_rate = tf.train.exponential_decay(0.5, global_step, 5000, 0.80, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights_input) + biases_input), weights_output) + biases_output)
  test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights_input) + biases_input), weights_output) + biases_output)

In [97]:
#Prasannas_Code
num_steps = 8001
with tf.Session(graph=graph) as session:
    
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):

    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("offset %d: %d" % (offset, offset + batch_size))
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
offset 0: 700
Minibatch loss at step 0: 1396.643799
Minibatch accuracy: 11.0%
Validation accuracy: 30.5%
offset 150700: 151400
Minibatch loss at step 500: 382.721375
Minibatch accuracy: 80.0%
Validation accuracy: 83.7%
offset 102100: 102800
Minibatch loss at step 1000: 226.724716
Minibatch accuracy: 80.3%
Validation accuracy: 84.8%
offset 53500: 54200
Minibatch loss at step 1500: 136.593307
Minibatch accuracy: 80.3%
Validation accuracy: 85.8%
offset 4900: 5600
Minibatch loss at step 2000: 82.299873
Minibatch accuracy: 83.9%
Validation accuracy: 86.6%
offset 155600: 156300
Minibatch loss at step 2500: 49.987354
Minibatch accuracy: 86.7%
Validation accuracy: 87.3%
offset 107000: 107700
Minibatch loss at step 3000: 30.489557
Minibatch accuracy: 84.1%
Validation accuracy: 87.9%
offset 58400: 59100
Minibatch loss at step 3500: 18.633158
Minibatch accuracy: 86.6%
Validation accuracy: 88.4%
offset 9800: 10500
Minibatch loss at step 4000: 11.465458
Minibatch accuracy: 88.4%
Validat